<a href="https://colab.research.google.com/github/EddiesCodes/Machine_learning/blob/master/CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Try CIFAR-10

Import librarys

In [0]:
import sys
import matplotlib.pyplot as plt
from keras.datasets import cifar10
if 'google.colab' in sys.modules:
    %tensorflow_version 2.x
import pandas as pd
import numpy as np
import tensorflow as tf
from pydrive.drive import GoogleDrive
from google_drive_downloader import GoogleDriveDownloader as gdd
from keras.utils.np_utils import to_categorical
from tensorflow.keras.layers import *

AUTO = tf.data.experimental.AUTOTUNE

##Configure the GPU

In [2]:
gpu = tf.config.experimental.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpu)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


##Download the data

In [3]:
(X, Y), (X_test, Y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [4]:
X.shape, Y.shape, X_test.shape, Y_test.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [5]:
np.unique(Y)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [0]:
y = to_categorical(Y.reshape(-1, 1), num_classes=10)
y_test = to_categorical(Y_test.reshape(-1, 1), num_classes=10)

In [0]:
def get_batched_dataset(dataset, train=False):
  if train:
    dataset = dataset.repeat()
    dataset = dataset.shuffle(1337)

  dataset = dataset.batch(BATCH_SIZE)
  return dataset

In [0]:
train = tf.data.Dataset.from_tensor_slices((X/255,y))
val = tf.data.Dataset.from_tensor_slices((X_test/255,y_test))

In [0]:
BATCH_SIZE = 64
steps_per_epoch = X.shape[0] // BATCH_SIZE
train = get_batched_dataset(train, train=True)
val = get_batched_dataset(val)

##Create resnet model

In [0]:
def conv_module(filter_size, x):
  y_1 = Conv2D(filter_size, kernel_size=3, strides=2)(x)
  y_1 = BatchNormalization()(y_1)
  y_1 = Activation('relu')(y_1)

  y_2 = Conv2D(filter_size, kernel_size=3, strides=2)(x)
  y_2 = BatchNormalization()(y_2)
  y_2 = Activation('relu')(y_2)

  y_3 = Conv2D(filter_size, kernel_size=1, strides=2)(x)
  y_3 = BatchNormalization()(y_1)
  y_3 = Activation('relu')(y_3)

  return Add()([y_1, y_2, y_3])

def conv_saufen(filter_size):
  return lambda x : conv_module(filter_size, x)

In [0]:
def identity_module(x):
  filter_size = x.shape[3]

  y_1 = Conv2D(filter_size, kernel_size=3, padding='same')(x)
  y_1 = BatchNormalization()(y_1)
  y_1 = Activation('relu')(y_1)
  y_1 = Conv2D(filter_size, kernel_size=3, padding='same')(y_1)
  y_1 = BatchNormalization()(y_1)
  y_1 = Activation('relu')(y_1)

  y_2 = Conv2D(filter_size, kernel_size=3, padding='same')(x)
  y_2 = BatchNormalization()(y_2)
  y_2 = Activation('relu')(y_2)
  y_2 = Conv2D(filter_size, kernel_size=3, padding='same')(y_2)
  y_2 = BatchNormalization()(y_2)
  y_2 = Activation('relu')(y_2)

  return Add()([y_1, y_2, x])

def id_saufen():
  return lambda x : identity_module(x)



In [0]:
def create_model():
  x = Input(shape = (32, 32, 3))

  y = Conv2D(32, 3)(x)
  y = Dropout(0.1)(y)

  y = id_saufen()(y)
  y = conv_saufen(64)(y)
  y = Dropout(0.15)(y)

  y = id_saufen()(y)
  y = conv_saufen(128)(y)
  y = Dropout(0.2)(y)

  y = id_saufen()(y)
  y = conv_saufen(256)(y)
  y = Dropout(0.25)(y)

  y = GlobalAveragePooling2D()(y)

  y = Dense(10, activation='softmax')(y)
  model = tf.keras.Model(x, y)
  return model

In [0]:
model_resnet = create_model()
model_resnet.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

In [0]:
model_resnet.summary()

In [0]:
annealer = tf.keras.callbacks.LearningRateScheduler(learn_rate)
model_resnet.fit(
          train,
          validation_data=val,
          epochs= 100,
          batch_size=BATCH_SIZE,
          steps_per_epoch = steps_per_epoch,
          callbacks=[annealer]
          
)

Epoch 1/100
781/781 [==============================] - 16s 21ms/step - loss: 1.5737 - accuracy: 0.4593 - val_loss: 1.4209 - val_accuracy: 0.5172 - lr: 0.0010
Epoch 2/100
781/781 [==============================] - 16s 21ms/step - loss: 0.9648 - accuracy: 0.6659 - val_loss: 1.0709 - val_accuracy: 0.6456 - lr: 0.0010
Epoch 3/100
781/781 [==============================] - 16s 21ms/step - loss: 0.7273 - accuracy: 0.7467 - val_loss: 0.9349 - val_accuracy: 0.6877 - lr: 0.0010
Epoch 4/100
781/781 [==============================] - 16s 21ms/step - loss: 0.5930 - accuracy: 0.7932 - val_loss: 1.0393 - val_accuracy: 0.6868 - lr: 0.0010
Epoch 5/100
781/781 [==============================] - 16s 21ms/step - loss: 0.5032 - accuracy: 0.8246 - val_loss: 0.6897 - val_accuracy: 0.7739 - lr: 0.0010
Epoch 6/100
781/781 [==============================] - 16s 21ms/step - loss: 0.4330 - accuracy: 0.8479 - val_loss: 0.8116 - val_accuracy: 0.7542 - lr: 0.0010
Epoch 7/100
781/781 [==============================]

In [0]:
def learn_rate(x):
  lr = 1e-3
  if x > 25:
    lr = lr*0.1
  return lr